## Initialise SparkSession and Load Data

In [1]:
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.local.dir', 'path')
sc = pyspark.SparkContext(conf=conf)

## Connection to Google BigQuery

In [2]:
from pythonbq import pythonbq

myProject=pythonbq(
  bq_key_path='json_file',
  project_id='project_id'
)

In [3]:
SQL_CODE="""
SELECT Country, MarketSegment, ArrivalDateMonth, DepositType, CustomerType, LeadTime, ArrivalDateYear, ArrivalDateWeekNumber, ArrivalDateDayOfMonth, RequiredCarParkingSpaces, IsCanceled FROM `table.H1data`
"""
output=myProject.query(sql=SQL_CODE)
output

,Country,MarketSegment,ArrivalDateMonth,DepositType,CustomerType,LeadTime,ArrivalDateYear,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,RequiredCarParkingSpaces,IsCanceled
0,PRT,Direct,July,No Deposit,Transient,342,2015,27,1,0,0
1,PRT,Direct,July,No Deposit,Transient,737,2015,27,1,0,0
2,GBR,Direct,July,No Deposit,Transient,7,2015,27,1,0,0
3,GBR,Corporate,July,No Deposit,Transient,13,2015,27,1,0,0
4,GBR,Online TA,July,No Deposit,Transient,14,2015,27,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
40055,GBR,Offline TA/TO,August,No Deposit,Transient,212,2017,35,31,0,0
40056,IRL,Direct,August,No Deposit,Transient-Party,169,2017,35,30,0,0
40057,IRL,Direct,August,No Deposit,Transient,204,2017,35,29,0,0
40058,GBR,Offline TA/TO,August,No Deposit,Contract,211,2017,35,31,0,0


In [5]:
# import pyspark
import pyarrow
import pandas as pd

In [6]:
type(output)

pandas.core.frame.DataFrame

In [7]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
dataset = sqlContext.createDataFrame(output)

# If reading from CSV:
# dataset= spark.read.load("H1full.csv", format="csv", header="true", inferSchema=True)

dataset
cols = dataset.columns

## Pipeline and Data Transformation

In [10]:
# https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
# https://docs.databricks.com/applications/machine-learning/mllib/binary-classification-mllib-pipelines.html

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ["Country", "MarketSegment", "ArrivalDateMonth", "DepositType", "CustomerType"]

stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

## Convert label into label indices using the StringIndexer

In [11]:
label_stringIdx = StringIndexer(inputCol="IsCanceled", outputCol="label")
stages += [label_stringIdx]

## Transform all features into a vector using VectorAssembler

In [12]:
numericCols = ["LeadTime", "ArrivalDateYear", "ArrivalDateWeekNumber", "ArrivalDateDayOfMonth", "RequiredCarParkingSpaces"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

## GBTClassifier

In [13]:
from pyspark.ml.classification import GBTClassifier
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [14]:
gbtClassifier = GBTClassifier()
trainedModel = gbtClassifier.fit(preppedDataDF)

In [ ]:
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)

## Train-Test Split

In [17]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed=100)
print(trainingData.count())
print(testData.count())

32101
7959


In [18]:
gbtModel = gbtClassifier.fit(trainingData)

In [ ]:
predictions = gbtModel.transform(testData)

In [20]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

## Model Evaluation

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.9123967272115422

In [22]:
evaluator.getMetricName()

'areaUnderROC'

In [24]:
predictions = gbtModel.transform(testData)

In [25]:
evaluator.evaluate(predictions)

0.9123967272115416

In [26]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

In [27]:
type(selected)

pyspark.sql.dataframe.DataFrame

In [28]:
selected.toPandas().to_csv('h1predictions.csv')

## Predictions on Test Data

In [29]:
mydata=pd.read_csv("h1predictions.csv")
mydata

,Unnamed: 0,label,prediction,probability
0,0,0.0,1.0,"[0.3328517247925773,0.6671482752074227]"
1,1,0.0,1.0,"[0.33385578033961344,0.6661442196603866]"
2,2,0.0,1.0,"[0.33385578033961344,0.6661442196603866]"
3,3,0.0,1.0,"[0.3295129995457048,0.6704870004542952]"
4,4,0.0,1.0,"[0.22810083419630225,0.7718991658036978]"
...,...,...,...,...
7954,7954,1.0,0.0,"[0.593856422149016,0.406143577850984]"
7955,7955,1.0,0.0,"[0.870767922996301,0.129232077003699]"
7956,7956,1.0,0.0,"[0.7108949915741888,0.2891050084258112]"
7957,7957,1.0,0.0,"[0.6716583759429362,0.3283416240570638]"


In [30]:
import numpy as np
from numpy.random import seed
seed(1)
from sklearn.metrics import classification_report,confusion_matrix

In [31]:
label=mydata['label']
prediction=mydata['prediction']

In [32]:
print(confusion_matrix(label,prediction))
print(classification_report(label,prediction))

[[5242  530]
 [ 752 1435]]
              precision    recall  f1-score   support

         0.0       0.87      0.91      0.89      5772
         1.0       0.73      0.66      0.69      2187

    accuracy                           0.84      7959
   macro avg       0.80      0.78      0.79      7959
weighted avg       0.83      0.84      0.84      7959

